# Dataset conversion from PASCAL VOC-like format to YOLO

Note: all file paths are set up for Kaggle. Output is 'visdrone-kau-yolo.zip'

In [ ]:
import os
from tqdm.notebook import tqdm
import shutil
from PIL import Image

In [3]:
path = "/kaggle/input/visdrone-kau-copy/visdrone_competition/train"   
write_path = "/kaggle/working/dataset/train"

In [ ]:
def process(path, write_path, test=False):
    def box_pixel_to_box_ratio(img_size, box_size):
        ratio_w = 1.0 / img_size[0]
        ratio_d = 1.0 / img_size[1]
        x = (box_size[0] + box_size[1]) / 2.0 - 1
        y = (box_size[2] + box_size[3]) / 2.0 - 1
        w = box_size[1] - box_size[0]
        h = box_size[3] - box_size[2]
        return [(x * ratio_w),
                (y * ratio_d),
                (w * ratio_w),
                (h * ratio_d),
                ]
        
        

    try:
        os.mkdir(write_path)
    except FileExistsError:
        ...
    try:
        os.mkdir(os.path.join(write_path, 'images'))
    except FileExistsError:
        ...
    shutil.copytree(os.path.join(path, 'images'), os.path.join(write_path, 'images'), dirs_exist_ok=True)
    if test:
        return
    items: list[str] = os.listdir(os.path.join(path,"labels_pascal"))
    ids = [x.replace(".txt", "") for x in items]
    try:
        os.mkdir(os.path.join(write_path, 'labels'))
    except FileExistsError:
        ...
    for id in tqdm(ids):
        size = Image.open(os.path.join(path, 'images', f'{id}.jpg')).size
    
        with open(os.path.join(path, 'labels_pascal', f'{id}.txt'), 'r') as f:
            lines = f.readlines()
        
        new_lines = []
        for line in lines:
            label, x_min, y_min, x_max, y_max = [int(x) for x in line.strip().split()]
            box = (x_min, x_max, y_min, y_max)
            converted_box = box_pixel_to_box_ratio(size, box)
            new_line = " ".join((f'{label}', *(str(x) for x in converted_box))) + '\n'
            new_lines.append(new_line)
        with open(os.path.join(write_path, 'labels', f'{id}.txt'), 'w') as f:
            f.writelines(new_lines)

In [5]:
try:
    os.mkdir("/kaggle/working/dataset/")
except:
    ...
for folder in tqdm(('train', 'val', 'test')):
    path = f"/kaggle/input/visdrone-kau-copy/visdrone_competition/{folder}"   
    write_path = f"/kaggle/working/dataset/{folder}"
    process(path, write_path, test=(folder == 'test'))
    

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6471 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

In [6]:
yaml =\
"""
path: /kaggle/input/visdrone-kau-2025-dataset-yolo-format/visdrone-kau-yolo
train: train/images
val: val/images
test: test/images

names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor
"""
with  open("/kaggle/working/dataset/visdrone.yaml", 'w') as f:
    f.write(yaml)

In [7]:
shutil.make_archive('visdrone-kau-yolo', 'zip', "/kaggle/working/dataset")

'/kaggle/working/visdrone-kau-yolo.zip'

In [8]:
shutil.rmtree('/kaggle/working/dataset')